In [1]:
#Imports

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.nn.parameter import Parameter
from spikingjelly.clock_driven.neuron import MultiStepLIFNode
from timm.models.layers import to_2tuple, trunc_normal_, DropPath
from timm.models.registry import register_model
from timm.models.vision_transformer import _cfg
from timm.utils import *
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchsummary import summary
from tqdm import tqdm, trange
import copy


#TODO: Define the model the spikeformer model
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1_linear = nn.Linear(in_features, hidden_features)
        self.fc1_bn = nn.BatchNorm1d(hidden_features)
        self.fc1_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

        self.fc2_linear = nn.Linear(hidden_features, out_features)
        self.fc2_bn = nn.BatchNorm1d(out_features)
        self.fc2_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

        self.c_hidden = hidden_features
        self.c_output = out_features

    def forward(self, x):
        T,B,N,C = x.shape
        x_ = x.flatten(0, 1)
        x = self.fc1_linear(x_)
        x = self.fc1_bn(x.transpose(-1, -2)).transpose(-1, -2).reshape(T, B, N, self.c_hidden).contiguous()
        x = self.fc1_lif(x)

        x = self.fc2_linear(x.flatten(0,1))
        x = self.fc2_bn(x.transpose(-1, -2)).transpose(-1, -2).reshape(T, B, N, C).contiguous()
        x = self.fc2_lif(x)
        return x


class SSA(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1):
        super().__init__()
        assert dim % num_heads == 0, f"dim {dim} should be divided by num_heads {num_heads}."
        self.dim = dim
        self.num_heads = num_heads
        self.scale = 0.125
        self.q_linear = nn.Linear(dim, dim)
        self.q_bn = nn.BatchNorm1d(dim)
        self.q_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

        self.k_linear = nn.Linear(dim, dim)
        self.k_bn = nn.BatchNorm1d(dim)
        self.k_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

        self.v_linear = nn.Linear(dim, dim)
        self.v_bn = nn.BatchNorm1d(dim)
        self.v_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')
        self.attn_lif = MultiStepLIFNode(tau=2.0, v_threshold=0.5, detach_reset=True, backend='torch')

        self.proj_linear = nn.Linear(dim, dim)
        self.proj_bn = nn.BatchNorm1d(dim)
        self.proj_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

    def forward(self, x):
        T,B,N,C = x.shape

        x_for_qkv = x.flatten(0, 1)  # TB, N, C
        q_linear_out = self.q_linear(x_for_qkv)  # [TB, N, C]
        # print("q_linear_out: ", q_linear_out.shape)
        q_linear_out = self.q_bn(q_linear_out. transpose(-1, -2)).transpose(-1, -2).reshape(T, B, N, C).contiguous()
        q_linear_out = self.q_lif(q_linear_out)
        q = q_linear_out.reshape(T, B, N, self.num_heads, C//self.num_heads).permute(0, 1, 3, 2, 4).contiguous()

        k_linear_out = self.k_linear(x_for_qkv)
        # print("k_linear_out after bn: ", k_linear_out.shape)
        k_linear_out = self.k_bn(k_linear_out. transpose(-1, -2)).transpose(-1, -2).reshape(T,B,C,N).contiguous()
        k_linear_out = self.k_lif(k_linear_out)
        k = k_linear_out.reshape(T, B, N, self.num_heads, C//self.num_heads).permute(0, 1, 3, 2, 4).contiguous()

        v_linear_out = self.v_linear(x_for_qkv)
        # print("v_linear_out after bn: ", v_linear_out.shape)
        v_linear_out = self.v_bn(v_linear_out. transpose(-1, -2)).transpose(-1, -2).reshape(T,B,C,N).contiguous()
        v_linear_out = self.v_lif(v_linear_out)
        v = v_linear_out.reshape(T, B, N, self.num_heads, C//self.num_heads).permute(0, 1, 3, 2, 4).contiguous()

        attn = (q @ k.transpose(-2, -1)) * self.scale
        x = attn @ v
        x = x.transpose(2, 3).reshape(T, B, N, C).contiguous()
        x = self.attn_lif(x)
        x = x.flatten(0, 1)
        x = self.proj_linear(x).transpose(-1, -2)
        # print("proj_norm_input", x.shape)
        x = self.proj_bn(x)
        x = self.proj_lif(x.transpose(-1, -2).reshape(T, B, N, C))
        return x

class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, sr_ratio=1):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = SSA(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                              attn_drop=attn_drop, proj_drop=drop, sr_ratio=sr_ratio)
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, drop=drop)

    def forward(self, x):
        x_attn = self.attn(x)
        x = x + x_attn
        x = x + self.mlp(x)

        return x


class SPS(nn.Module):
    def __init__(self, img_size_h=128, img_size_w=128, patch_size=4, in_channels=2, embed_dims=256):
        super().__init__()
        self.image_size = [img_size_h, img_size_w]
        patch_size = to_2tuple(patch_size)
        self.patch_size = patch_size
        self.C = in_channels
        self.H, self.W = self.image_size[0] // patch_size[0], self.image_size[1] // patch_size[1]
        self.num_patches = self.H * self.W
        self.proj_conv = nn.Conv2d(in_channels, embed_dims//8, kernel_size=3, stride=1, padding=1, bias=False)
        self.proj_bn = nn.BatchNorm2d(embed_dims//8)
        self.proj_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

        self.proj_conv1 = nn.Conv2d(embed_dims//8, embed_dims//4, kernel_size=3, stride=1, padding=1, bias=False)
        self.proj_bn1 = nn.BatchNorm2d(embed_dims//4)
        self.proj_lif1 = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

        self.proj_conv2 = nn.Conv2d(embed_dims//4, embed_dims//2, kernel_size=3, stride=1, padding=1, bias=False)
        self.proj_bn2 = nn.BatchNorm2d(embed_dims//2)
        self.proj_lif2 = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)

        self.proj_conv3 = nn.Conv2d(embed_dims//2, embed_dims, kernel_size=3, stride=1, padding=1, bias=False)
        self.proj_bn3 = nn.BatchNorm2d(embed_dims)
        self.proj_lif3 = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')
        self.maxpool3 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)

        self.rpe_conv = nn.Conv2d(embed_dims, embed_dims, kernel_size=3, stride=1, padding=1, bias=False)
        self.rpe_bn = nn.BatchNorm2d(embed_dims)
        self.rpe_lif = MultiStepLIFNode(tau=2.0, detach_reset=True, backend='torch')

    def forward(self, x):
        T, B, C, H, W = x.shape
        x = self.proj_conv(x.flatten(0, 1)) # have some fire value
        x = self.proj_bn(x).reshape(T, B, -1, H, W).contiguous()
        x = self.proj_lif(x).flatten(0, 1).contiguous()

        x = self.proj_conv1(x)
        x = self.proj_bn1(x).reshape(T, B, -1, H, W).contiguous()
        x = self.proj_lif1(x).flatten(0, 1).contiguous()

        x = self.proj_conv2(x)
        x = self.proj_bn2(x).reshape(T, B, -1, H, W).contiguous()
        x = self.proj_lif2(x).flatten(0, 1).contiguous()
        x = self.maxpool2(x)

        x = self.proj_conv3(x)
        x = self.proj_bn3(x).reshape(T, B, -1, H//2, W//2).contiguous()
        x = self.proj_lif3(x).flatten(0, 1).contiguous()
        x = self.maxpool3(x)

        x_feat = x.reshape(T, B, -1, H//4, W//4).contiguous()
        x = self.rpe_conv(x)
        x = self.rpe_bn(x).reshape(T, B, -1, H//4, W//4).contiguous()
        x = self.rpe_lif(x)
        x = x + x_feat

        x = x.flatten(-2).transpose(-1, -2)  # T,B,N,C
        return x


class Spikformer(nn.Module):
    def __init__(self,
                 img_size_h=32, img_size_w=32, patch_size=4, in_channels=3, num_classes=10,
                 embed_dims=384, num_heads=12, mlp_ratios=4, qkv_bias=False, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm,
                 depths=4, sr_ratios=1, T = 4
                 ):
        super().__init__()
        self.T = T  # time step
        self.num_classes = num_classes
        self.depths = depths

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depths)]  # stochastic depth decay rule

        patch_embed = SPS(img_size_h=img_size_h,
                                 img_size_w=img_size_w,
                                 patch_size=patch_size,
                                 in_channels=in_channels,
                                 embed_dims=embed_dims)

        block = nn.ModuleList([Block(
            dim=embed_dims, num_heads=num_heads, mlp_ratio=mlp_ratios, qkv_bias=qkv_bias,
            qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[j],
            norm_layer=norm_layer, sr_ratio=sr_ratios)
            for j in range(depths)])

        setattr(self, f"patch_embed", patch_embed)
        setattr(self, f"block", block)

        # classification head
        self.head = nn.Linear(embed_dims, num_classes) if num_classes > 0 else nn.Identity()
        self.apply(self._init_weights)

    @torch.jit.ignore
    def _get_pos_embed(self, pos_embed, patch_embed, H, W):
        if H * W == self.patch_embed1.num_patches:
            return pos_embed
        else:
            return F.interpolate(
                pos_embed.reshape(1, patch_embed.H, patch_embed.W, -1).permute(0, 3, 1, 2),
                size=(H, W), mode="bilinear").reshape(1, -1, H * W).permute(0, 2, 1)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):

        block = getattr(self, f"block")
        patch_embed = getattr(self, f"patch_embed")

        x = patch_embed(x)
        for blk in block:
            x = blk(x)
        return x.mean(2)

    def forward(self, x):
        x = (x.unsqueeze(0)).repeat(self.T, 1, 1, 1, 1)
        x = self.forward_features(x)
        x = self.head(x.mean(0))
        return x

    
#Import CIFAR datasets

# dataloader arguments
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Define a transform
transform = transforms.Compose([
            transforms.ToTensor()])

batch_size = 128

CIFAR_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
CIFAR_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
classes = CIFAR_train.classes

#reduce datasize
# subsets_train = list(range(0, len(CIFAR_train), 10))
# subsets_test = list(range(0, len(CIFAR_test), 10))

# CIFAR_train = torch.utils.data.Subset(CIFAR_train, subsets_train)
# CIFAR_test = torch.utils.data.Subset(CIFAR_test, subsets_test)

train_loader = DataLoader(CIFAR_train,batch_size=batch_size,shuffle=True, drop_last=True)
test_loader = DataLoader(CIFAR_test,batch_size=batch_size,shuffle=True, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
# def accuracy(Y, targets):
#     Y = torch.argmax(Y, axis=1)
#     total = Y.shape[0]
#     return torch.sum(Y==targets).detach().cpu().item()/total

In [3]:
def batch_accuracy(train_loader, net):
    with torch.no_grad():
        total = 0
        acc = 0
        net.eval()
        
        train_loader = iter(train_loader)
        for batch in tqdm(train_loader):
            data, targets = batch
            data = data.to(device)
            targets = targets.to(device)
            
            out = net(data)
            acc1, acc5 = accuracy(out, targets, topk=(1,5))
            
    return acc1, acc5


In [4]:
# Load trained model 
net = Spikformer()
checkpoint_path = "/Volumes/export/isn/gopa/CRI_proj/cri_spikeformer/cifar10/output/train/20230301-194515-spikformer-32/checkpoint-307.pth.tar"
checkpoint = torch.load(checkpoint_path)
net.load_state_dict(checkpoint['state_dict'])


<All keys matched successfully>

In [5]:
# if torch.cuda.is_available():
#     net.cuda()
# batch_accuracy(test_loader,net)

In [6]:
summary(net)

Layer (type:depth-idx)                        Param #
├─SPS: 1-1                                    --
|    └─Conv2d: 2-1                            1,296
|    └─BatchNorm2d: 2-2                       96
|    └─MultiStepLIFNode: 2-3                  --
|    |    └─Sigmoid: 3-1                      --
|    └─Conv2d: 2-4                            41,472
|    └─BatchNorm2d: 2-5                       192
|    └─MultiStepLIFNode: 2-6                  --
|    |    └─Sigmoid: 3-2                      --
|    └─Conv2d: 2-7                            165,888
|    └─BatchNorm2d: 2-8                       384
|    └─MultiStepLIFNode: 2-9                  --
|    |    └─Sigmoid: 3-3                      --
|    └─MaxPool2d: 2-10                        --
|    └─Conv2d: 2-11                           663,552
|    └─BatchNorm2d: 2-12                      768
|    └─MultiStepLIFNode: 2-13                 --
|    |    └─Sigmoid: 3-4                      --
|    └─MaxPool2d: 2-14                      

Layer (type:depth-idx)                        Param #
├─SPS: 1-1                                    --
|    └─Conv2d: 2-1                            1,296
|    └─BatchNorm2d: 2-2                       96
|    └─MultiStepLIFNode: 2-3                  --
|    |    └─Sigmoid: 3-1                      --
|    └─Conv2d: 2-4                            41,472
|    └─BatchNorm2d: 2-5                       192
|    └─MultiStepLIFNode: 2-6                  --
|    |    └─Sigmoid: 3-2                      --
|    └─Conv2d: 2-7                            165,888
|    └─BatchNorm2d: 2-8                       384
|    └─MultiStepLIFNode: 2-9                  --
|    |    └─Sigmoid: 3-3                      --
|    └─MaxPool2d: 2-10                        --
|    └─Conv2d: 2-11                           663,552
|    └─BatchNorm2d: 2-12                      768
|    └─MultiStepLIFNode: 2-13                 --
|    |    └─Sigmoid: 3-4                      --
|    └─MaxPool2d: 2-14                      

In [7]:
#TODO: check the correctness of folding 
class BN_Folder():
    def __init__(self):
        super().__init__()
        
    def fold(self, model):

        new_model = copy.deepcopy(model)

        module_names = list(new_model._modules)

        for k, name in enumerate(module_names):

            if len(list(new_model._modules[name]._modules)) > 0:
                
                new_model._modules[name] = self.fold(new_model._modules[name])

            else:
                if isinstance(new_model._modules[name], nn.BatchNorm2d):
                    if isinstance(new_model._modules[module_names[k-1]], nn.Conv2d):

                        # Folded BN
                        folded_conv = self._fold_conv_bn_eval(new_model._modules[module_names[k-1]], new_model._modules[name])

                        # Replace old weight values
                        #new_model._modules.pop(name) # Remove the BN layer
                        new_model._modules[module_names[k]] = nn.Identity()
                        new_model._modules[module_names[k-1]] = folded_conv # Replace the Convolutional Layer by the folded version
                        
                if isinstance(new_model._modules[name], nn.BatchNorm1d):
                    if isinstance(new_model._modules[module_names[k-1]], nn.Linear) :
                        # Folded BN
                        folded_conv = self._fold_conv_bn_eval(new_model._modules[module_names[k-1]], new_model._modules[name])

                        # Replace old weight values
                        #new_model._modules.pop(name) # Remove the BN layer
                        new_model._modules[module_names[k]] = nn.Identity()
                        new_model._modules[module_names[k-1]] = folded_conv # Replace the Convolutional Layer by the folded version

        return new_model


    def _bn_folding(self, conv_w, conv_b, bn_rm, bn_rv, bn_eps, bn_w, bn_b):
        if conv_b is None:
            conv_b = bn_rm.new_zeros(bn_rm.shape)
        bn_var_rsqrt = torch.rsqrt(bn_rv + bn_eps)

        w_fold = conv_w * (bn_w * bn_var_rsqrt).view(-1, 1, 1, 1)
        b_fold = (conv_b - bn_rm) * bn_var_rsqrt * bn_w + bn_b

        return torch.nn.Parameter(w_fold), torch.nn.Parameter(b_fold)


    def _bn_folding_linear(self, conv_w, conv_b, bn_rm, bn_rv, bn_eps, bn_w, bn_b):
        if conv_b is None:
            conv_b = bn_rm.new_zeros(bn_rm.shape)
        
        bn_var_rsqrt = torch.rsqrt(bn_rv + bn_eps)

        print("Before: ",conv_w.shape, conv_b.shape)
        w_fold = conv_w * (bn_w * bn_var_rsqrt).view(-1, 1)
        b_fold = (conv_b - bn_rm) * bn_var_rsqrt * bn_w + bn_b
        print("After: ",w_fold.shape, b_fold.shape)
        
        return torch.nn.Parameter(w_fold), torch.nn.Parameter(b_fold)

        
    def _fold_conv_bn_eval(self, conv, bn):
        assert(not (conv.training or bn.training)), "Fusion only for eval!"
        fused_conv = copy.deepcopy(conv)
        
        if isinstance(bn, nn.BatchNorm1d):
            fused_conv.weight, fused_conv.bias = self._bn_folding_linear(fused_conv.weight, fused_conv.bias,
                                     bn.running_mean, bn.running_var, bn.eps, bn.weight, bn.bias)
        else:
            fused_conv.weight, fused_conv.bias = self._bn_folding(fused_conv.weight, fused_conv.bias,
                                     bn.running_mean, bn.running_var, bn.eps, bn.weight, bn.bias)

        return fused_conv
     

In [8]:
net.eval()

bn = BN_Folder()
model = bn.fold(net)

summary(model)

Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([1536, 384]) torch.Size([1536])
After:  torch.Size([1536, 384]) torch.Size([1536])
Before:  torch.Size([384, 1536]) torch.Size([384])
After:  torch.Size([384, 1536]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Before:  torch.Size([384, 384]) torch.Size([384])
After:  torch.Size([384, 384]) torch.Size([384])
Befo

Layer (type:depth-idx)                        Param #
├─SPS: 1-1                                    --
|    └─Conv2d: 2-1                            1,344
|    └─Identity: 2-2                          --
|    └─MultiStepLIFNode: 2-3                  --
|    |    └─Sigmoid: 3-1                      --
|    └─Conv2d: 2-4                            41,568
|    └─Identity: 2-5                          --
|    └─MultiStepLIFNode: 2-6                  --
|    |    └─Sigmoid: 3-2                      --
|    └─Conv2d: 2-7                            166,080
|    └─Identity: 2-8                          --
|    └─MultiStepLIFNode: 2-9                  --
|    |    └─Sigmoid: 3-3                      --
|    └─MaxPool2d: 2-10                        --
|    └─Conv2d: 2-11                           663,936
|    └─Identity: 2-12                         --
|    └─MultiStepLIFNode: 2-13                 --
|    |    └─Sigmoid: 3-4                      --
|    └─MaxPool2d: 2-14                        -

In [ ]:
# #Test the accuracy
# if torch.cuda.is_available():
#     model.cuda()
# batch_accuracy(test_loader,model)

In [ ]:
#TODO: Mapping to CRI